<a href="https://colab.research.google.com/github/cyrus2281/notes/blob/main/MachineLearning/Generative_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content

>[Content](#scrollTo=9CWyV4beAH0q&uniqifier=1)

>[Generative AI](#scrollTo=d95n8HFhAJ3m&uniqifier=1)

>>[OpenAI](#scrollTo=i4ToZgP9-rIo&uniqifier=1)

>>>[Few shot Prompt](#scrollTo=wW5Py1HkneoI&uniqifier=1)

>>>[Function Calling](#scrollTo=_sHjRxqRu1SE&uniqifier=1)

>>[LangChain](#scrollTo=VaZgHOlOly0L&uniqifier=1)

>>>[Prompt Templating](#scrollTo=Qrmc6aYCmyBE&uniqifier=1)

>>>[Agents](#scrollTo=OSJlfKhYm05y&uniqifier=1)

>>>[Chains](#scrollTo=wZB8-VEzm0qW&uniqifier=1)

>>>[Document Loader](#scrollTo=094HkTgJ3LgE&uniqifier=1)

>>>[Memory](#scrollTo=1yM7479RABo5&uniqifier=1)

>>>>[Conversation Buffer Memory](#scrollTo=xUDLj9bZDX1H&uniqifier=1)

>>>>[Conversation Chain](#scrollTo=34Wx_PAVHJZc&uniqifier=1)

>>>>[Conversation Buffer Window Memory](#scrollTo=bE0qBhKn1sfW&uniqifier=1)

>>[Hugging Face](#scrollTo=4J6b25uaB5TP&uniqifier=1)

>>>[Using Models from HuggingFace](#scrollTo=Pq3G4sxRDmAH&uniqifier=1)

>>>[Downloading Models Locally](#scrollTo=9VtpZ9UlMPlr&uniqifier=1)

>>[Vector Databases](#scrollTo=3TY5pCv8w-5H&uniqifier=1)

>>>[Pinecone VectorDB](#scrollTo=fX0221ouxAfx&uniqifier=1)

>>>>[Creating Embeddings from text chunks](#scrollTo=ChlWuiNPB4er&uniqifier=1)

>>>>[Querying VectorDB](#scrollTo=WtglMMDjKD-i&uniqifier=1)

>>>[ChromaDB](#scrollTo=kD8z-5adnZ4T&uniqifier=1)

>>>>[Creating ChromaDB](#scrollTo=glV_-rzq8mBE&uniqifier=1)

>>>>[Retrieval](#scrollTo=EtSO9IhK-iZC&uniqifier=1)

>>>>[Making a Chain](#scrollTo=Rz1W-meADHuS&uniqifier=1)

>>>>[Removing ChromaDB](#scrollTo=nTnNTxUCHD2o&uniqifier=1)

>>[Llama](#scrollTo=cIbus3vrIFjB&uniqifier=1)

>>>[Llama CPP Python](#scrollTo=KwSz42p_TGN4&uniqifier=1)

>>>[Llama with Langchain](#scrollTo=U_LQmLSHMnoO&uniqifier=1)



# Generative AI

## OpenAI

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
import openai
print(openai.__version__)

1.14.1


In [ ]:
openai_api_key = "" # @param {type:"string"}

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ], # Roles: system, user, assistant, tool
  max_tokens=128, # Maximum number of tokens you want result
  temperature=1, # How creative the output should be
  n=2 # Number of the outputs
)

print(completion.choices[0].message)

ChatCompletionMessage(content='In the realm of code, a concept lives and breathes,\nWhere functions call themselves, like whispers in the trees.\nRecursion is the art, a trance-like dance,\nUnfolding mysteries with each recursive glance.\n\nLike a mirror reflecting its own reflection,\nA function loops back, with no objection.\nIt breaks down a problem, into smaller parts,\nAnd solves them one by one, with gentle smarts.\n\nA journey through layers, like a dream unfurled,\nEach invocation a thread in the code world.\nIt reaches the base case, like a destination,\nUnwinding the stack, with jubilation.\n\nSo embrace recursion, with courage and grace,\nLet it guide your code through time and space.\nFor in its elegant cycle, we find our place,\nA recursive enchantment, woven with grace.', role='assistant', function_call=None, tool_calls=None)


**This is an example of zero shot prompt**

### Few shot Prompt


In [ ]:
person_info="My name is Cyrus, I'm a software engineer at X, and I live in Canada."

In [ ]:
prompt = f'''
Extract the following information from the given text and return it as a JSON object:

name
job
company
country

This is the body of text to extract the information from:
{person_info}
'''

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": prompt}
  ]
)

print(completion.choices[0].message.content)

{
  "name": "Cyrus",
  "job": "software engineer",
  "company": "X",
  "country": "Canada"
}


**This is an example of few shot prompt**

### Function Calling

In [ ]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


In [ ]:
# Step 1: send the conversation and available functions to the model
messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
response_message = response.choices[0].message
tool_calls = response_message.tool_calls

In [ ]:
# Step 2: check if the model wanted to call a function
if tool_calls:
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }  # only one function in this example, but you can have multiple
    messages.append(response_message)  # extend conversation with assistant's reply
    # Step 4: send the info for each function call and function response to the model
    for tool_call in tool_calls:
        print(tool_call)
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
    )  # get a new response from the model where it can see the function response
    print(second_response.choices[0].message.content)

ChatCompletionMessageToolCall(id='call_Q6tNs8Yd24ROxqmgrMYYYAC1', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function')
ChatCompletionMessageToolCall(id='call_efCZ1xEyMhMqKCVvFNsU3gax', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function')
ChatCompletionMessageToolCall(id='call_UgPOHn0Gls5GEtPiNdYlseoB', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')
Currently in San Francisco, the weather is 72°C. In Tokyo, it is 10°C, and in Paris, it is 22°C.


## LangChain

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 21.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
import langchain
print(langchain.__version__)

0.1.12


In [ ]:
from langchain.llms import OpenAI
client=OpenAI(openai_api_key=openai_api_key)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
# Zero shot prompting
prompt="Can you tell me total number of counntries in north america"
print(client.invoke(prompt))



There are 23 countries in North America. They are:

1. Antigua and Barbuda
2. Bahamas
3. Barbados
4. Belize
5. Canada
6. Costa Rica
7. Cuba
8. Dominica
9. Dominican Republic
10. El Salvador
11. Grenada
12. Guatemala
13. Haiti
14. Honduras
15. Jamaica
16. Mexico
17. Nicaragua
18. Panama
19. Saint Kitts and Nevis
20. Saint Lucia
21. Saint Vincent and the Grenadines
22. Trinidad and Tobago
23. United States of America


### Prompt Templating

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["country"],
    template="Can you tell me the capital {country}"
)
prompt_template

PromptTemplate(input_variables=['country'], template='Can you tell me the capital {country}')

In [ ]:
prompt = prompt_template.format(country="Canada")

In [ ]:
print(client.invoke(prompt))



The capital of Canada is Ottawa.


In [ ]:
prompt_template = PromptTemplate.from_template("What is the language of {country}")

In [ ]:
prompt_template.format(country="Canada")

'What is the language of Canada'

### Agents

Used to call third-party tools

In [ ]:
prompt_template = PromptTemplate.from_template("What is the weather in {city} now?")
prompt=prompt_template.format(city="Ottawa")

In [ ]:
print(client.invoke(prompt)) # Not a good answer



I am an AI and do not have access to real-time weather data. Please check a weather website or app for the current weather in Ottawa.


In [ ]:
!pip install google-search-results

In [ ]:
serp_api_key = "" # @param {type:"string"}


In [ ]:
from langchain.agents import AgentType, load_tools, initialize_agent

In [ ]:
tool = load_tools(["serpapi"], serpapi_api_key=serp_api_key, llm=client)
tool

[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='46f2201c74213cb9e5f88667f9bc98ea9b2ce2343d5cc3a206bc5c67c9c85b33', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='46f2201c74213cb9e5f88667f9bc98ea9b2ce2343d5cc3a206bc5c67c9c85b33', aiosession=None)>)]

In [ ]:
agent=initialize_agent(
    tool,
    client,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
agent.run(prompt)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use the search engine to find the current weather in Ottawa
Action: Search
Action Input: "weather in Ottawa"
Observation: {'type': 'weather_result', 'temperature': '33', 'unit': 'Fahrenheit', 'precipitation': '19%', 'humidity': '91%', 'wind': '8 mph', 'location': 'Ottawa, ON, Canada', 'date': 'Tuesday 11:00 PM', 'weather': 'Cloudy'}
Thought: The current weather in Ottawa is cloudy with a temperature of 33 degrees Fahrenheit.
Final Answer: The current weather in Ottawa is cloudy with a temperature of 33 degrees Fahrenheit.

> Finished chain.


'The current weather in Ottawa is cloudy with a temperature of 33 degrees Fahrenheit.'

### Chains

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step.

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template1 = PromptTemplate.from_template("What is a good name for a {obj} bussiness? (Only one example)")
prompt_template2 = PromptTemplate.from_template("Creating a startup with the name {name} , give me ideas?")

In [ ]:
from langchain.chains import LLMChain

name_chain = LLMChain(llm=client, prompt=prompt_template1, output_key="name")

In [ ]:
print(name_chain.run("flower"))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




"Blooming Blossoms" 


In [ ]:
idea_chain = LLMChain(llm=client, prompt=prompt_template2, output_key="ideas")

In [ ]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[name_chain, idea_chain])

In [ ]:
print(chain.run("flower"))



1. A unique and personalized floral subscription service, where customers can receive a fresh bouquet of flowers every week or month.

2. Offering floral arrangements for special events such as weddings, birthdays, and corporate events.

3. Partnering with local businesses to provide floral arrangements for their offices or storefronts.

4. Hosting workshops and classes on floral arrangement and design for individuals or groups.

5. Developing a line of eco-friendly and sustainable floral products, such as biodegradable flower pots and packaging.

6. Providing flower delivery services for last-minute gifts or special occasions.

7. Collaborating with local artists to create unique and one-of-a-kind floral art installations for public spaces.

8. Offering a selection of dried and preserved flowers for long-lasting home decor.

9. Creating a social media presence to showcase the beauty of flowers and engage with customers.

10. Partnering with local flower farms to source fresh, locall

**Alternative syntax**

In [ ]:
chain = prompt_template1 | client

print(chain.invoke({"obj": "flower bussiness"}))



"Petals & Blooms Co."


In [ ]:
def str_parser(input):
  print(input)
  return {"name": input}

chain = prompt_template1 | client | str_parser | prompt_template2 | client
print(chain.invoke({"obj": "flower bussiness"}))



"Blooming Oasis Florals"


1. Unique floral arrangements: Offer a variety of unique and creative floral arrangements, such as succulent bouquets, preserved flower displays, and personalized floral designs.

2. Subscription services: Provide a subscription service for weekly or monthly flower deliveries to homes and businesses, offering a convenient and cost-effective option for customers.

3. Event and wedding services: Partner with event planners and wedding venues to offer floral design services for special occasions, including customized centerpieces, bouquets, and installations.

4. Online presence: Create a strong online presence through a website and social media platforms to showcase your work, promote your services, and interact with customers.

5. Green and sustainable practices: Promote eco-friendly and sustainable practices, such as using locally sourced flowers and recyclable packaging, to appeal to environmentally conscious customers.

6. Collaborations: Collaborate with

Sequential Chains

In [ ]:
from langchain.chains import SequentialChain

chain=SequentialChain(
    chains=[name_chain, idea_chain],
    input_variables=["obj"],
    output_variables=["name", "ideas"]
    )

In [ ]:
chain({"obj": "flower"})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'obj': 'flower',
 'name': '\n\n"Petals & Blooms Floral Co." ',
 'ideas': ' \n\n1. Online Flower Delivery Service: Offer a convenient and easy way for customers to order and send flowers to their loved ones with same-day delivery options.\n\n2. Custom Floral Arrangements: Create unique and personalized floral arrangements for special occasions such as weddings, birthdays, and anniversaries.\n\n3. Subscription Services: Offer a subscription service where customers can receive a fresh bouquet of flowers every week or month.\n\n4. Event Planning and Decor: Partner with event planners and offer floral decor for weddings, corporate events, and other special occasions.\n\n5. Floral Workshops: Host workshops for customers to learn how to create their own floral arrangements and bouquets.\n\n6. Greenhouse and Nursery: Have a greenhouse and nursery on-site where customers can purchase plants and flowers for their home or garden.\n\n7. Corporate Gifting: Partner with businesses to provide custom

### Document Loader

In [ ]:
!pip install pypdf langchain_openai  faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.9 MB/s eta 0:00:00


In [ ]:
!wget https://papers.neurips.cc/paper/7181-attention-is-all-you-need.pdf

--2024-03-21 02:42:10--  https://papers.neurips.cc/paper/7181-attention-is-all-you-need.pdf
Resolving papers.neurips.cc (papers.neurips.cc)... 198.202.70.94
Connecting to papers.neurips.cc (papers.neurips.cc)|198.202.70.94|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf [following]
--2024-03-21 02:42:10--  https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf
Resolving proceedings.neurips.cc (proceedings.neurips.cc)... 198.202.70.94
Connecting to proceedings.neurips.cc (proceedings.neurips.cc)|198.202.70.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 569417 (556K) [application/pdf]
Saving to: ‘7181-attention-is-all-you-need.pdf’

7181-attention-is-a 100%[===================>] 556.07K  1.69MB/s    in 0.3s    

2024-03-21 02:42:11 (1.69 MB/s) - ‘7181-attention-is-all-you-need.pdf’

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/7181-attention-is-all-you-need.pdf")
pages = loader.load_and_split()

In [ ]:
len(pages), pages[0]

(12,
 Document(page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requ

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages[0:3], OpenAIEmbeddings(api_key=openai_api_key))

In [ ]:
docs = faiss_index.similarity_search("What is attention?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

0: Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗†
University of Toronto
aidan@c
1: Recurrent models typically factor computation along the symbol positions of the input and output
sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden
statesht, as a function of the previous hidden state ht−1and the input for position t. This inherently
s


### Memory

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template("What is a good name for a {obj} bussiness? (Only one example)")

In [ ]:
from langchain.chains import LLMChain
chain = LLMChain(llm=client, prompt=prompt_template)

In [ ]:
chain.run("AI").strip()

'"Synapse Solutions"'

In [ ]:
chain.memory

In [ ]:
type(chain.memory)

NoneType

#### Conversation Buffer Memory

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

Manually adding to memory

```python
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")
```

In [ ]:
chain = LLMChain(llm=client, prompt=prompt_template, memory=memory)

In [ ]:
chain.run("AI").strip()

'"NeuroNexus Solutions"'

In [ ]:
chain.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='AI'), AIMessage(content='\n\n"NeuroNexus Solutions"')]))

In [ ]:
print(chain.memory.buffer)

Human: AI
AI: 

"NeuroNexus Solutions"


#### Conversation Chain

Conversation buffer memory goes growing endlessly.

Restrict to last 5 conversation chain

In [ ]:
from langchain.chains import ConversationChain
from langchain_openai import OpenAI

In [ ]:
convo = ConversationChain(llm=OpenAI(openai_api_key=openai_api_key, temperature=0.7))

In [ ]:
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [ ]:
convo.run("Who invented the light bulb?")

' The light bulb was invented by Thomas Edison in 1879. However, he was not the only person working on this technology at the time. Other inventors like Joseph Swan and Hiram Maxim were also working on improving the design and functionality of the light bulb.'

In [ ]:
convo.run("When did he born?")

' Thomas Edison was born on February 11, 1847 in Milan, Ohio. He was the youngest of seven children and showed an interest in science and technology from a young age.'

#### Conversation Buffer Window Memory




In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory=ConversationBufferWindowMemory(k=1)

In [ ]:
convo = ConversationChain(llm=OpenAI(openai_api_key=openai_api_key, temperature=0.7), memory=memory)

In [ ]:
convo.run("Who invented the light bulb?")

' The light bulb was invented by Thomas Edison in 1879. He experimented with over 6,000 different materials before discovering that carbonized bamboo worked best as a filament.'

In [ ]:
convo.run("What year is today?")

' Today is Monday, December 13, 2021.'

In [ ]:
convo.run("When did he born?")

' I do not have enough information to accurately answer your question. Who are you referring to?'

## Hugging Face

In [ ]:
!pip install huggingface_hub transformers accelerate bitsandbytes langchain

In [ ]:
!pip install langchain

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
import os

In [ ]:
HUGGINGFACEHUB_API_TOKEN = "" # @param {type:"string"}


In [ ]:
os.environ['HUGGINGFACEHUB_API_TOKEN']=HUGGINGFACEHUB_API_TOKEN

### Using Models from HuggingFace

In [ ]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that make {product}"
)

In [ ]:
chain = LLMChain(
    llm=HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={'temperature': 0}),
    prompt=prompt
  )

In [ ]:
chain.run("flower")

'lilies'

### Downloading Models Locally

In [ ]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

In [ ]:
model_id="google/flan-t5-large"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
pline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=128)

In [ ]:
local_llm = HuggingFacePipeline(pipeline=pline)

In [ ]:
prompt = PromptTemplate(
    input_variables=["item"],
    template="Tell me about {item}"
)

In [ ]:
chain = LLMChain(llm=local_llm, prompt=prompt)

In [ ]:
chain("Sun")

{'item': 'Sun',
 'text': 'Sun is a constellation in the constellation of Aries.'}

## Vector Databases

### Pinecone VectorDB

In [ ]:
!pip install langchain pinecone-client pypdf openai tiktoken langchain_pinecone langchain-openai

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
import os

In [ ]:
!mkdir pdfs

In [ ]:
!wget https://papers.neurips.cc/paper/7181-attention-is-all-you-need.pdf
!mv /content/7181-attention-is-all-you-need.pdf /content/pdfs/attention-is-all-you-need.pdf

In [ ]:
loader = PyPDFDirectoryLoader('pdfs')

In [ ]:
data = loader.load()
len(data), data[0]

(11,
 Document(page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requ

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [ ]:
text_chunks = text_splitter.split_documents(data)

In [ ]:
len(text_chunks)

76

In [ ]:
print(text_chunks[0].page_content)

Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or


In [ ]:
print(text_chunks[1].page_content)

convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring signiﬁcantly


In [ ]:
openai_api_key = "" # @param {type:"string"}

In [ ]:
embedding = OpenAIEmbeddings(api_key=openai_api_key)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
embedding_example = embedding.embed_query("Hello World")
len(embedding_example), embedding_example[:15]

(1536,
 [-0.007116983617481661,
  0.0034469901452434362,
  -0.007148840903008569,
  -0.029156057052714266,
  -0.013061608137966748,
  0.010946264448676999,
  -0.020274164077690417,
  0.0052533149518593026,
  -0.008563317325787047,
  -0.03017549950280105,
  0.024390162806934363,
  0.009863106496213825,
  -0.027524949877633466,
  -0.006632748500256566,
  0.009111268411049814])

In [ ]:
pinecone_api_key = "" # @param {type:"string"}
pinecone_api_env = "" # @param {type:"string"}

In [ ]:
from pinecone import Pinecone
pc = Pinecone(
    api_key=pinecone_api_key,
    environment=pinecone_api_env
)

In [ ]:
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'genai-course-iqutuem.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'genai-course',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

#### Creating Embeddings from text chunks

In [ ]:
text_chunks[0].page_content

'Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or'

In [ ]:
index_name = "genai-course"
os.environ["PINECONE_API_KEY"] = pinecone_api_key
os.environ["PINECONE_INDEX_NAME"] = index_name

In [ ]:
index_name = "genai-course"

docsearch = PineconeVectorStore.from_documents(
    text_chunks,
    embedding,
    index_name=index_name,
)

#### Querying VectorDB

In [ ]:
query = "What are decoder components"

In [ ]:
docs = docsearch.similarity_search(query)

In [ ]:
len(docs), docs[0].page_content

(4,
 'Decoder: The decoder is also composed of a stack of N= 6identical layers. In addition to the two\nsub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head\nattention over the output of the encoder stack. Similar to the encoder, we employ residual connections\naround each of the sub-layers, followed by layer normalization. We also modify the self-attention\nsub-layer in the decoder stack to prevent positions from attending to subsequent positions. This')

Manually decoding embeded vectors

In [ ]:
llm = OpenAI(api_key=openai_api_key)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

In [ ]:
qa.run(query)

' The decoder components include a stack of N= 6 identical layers, with a third sub-layer for multi-head attention over the output of the encoder stack. The decoder also uses residual connections and layer normalization, and modifies the self-attention sub-layer to prevent positions from attending to subsequent positions. Additionally, the decoder follows the same overall architecture as the encoder, using stacked self-attention and point-wise, fully connected layers. '

In [ ]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x7bf5a8303a90>)

Using Pinecone directly

In [ ]:
index = pc.Index(index_name)

In [ ]:
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'genai-course-iqutuem.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'genai-course',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [ ]:
embeded_query = embedding.embed_query(query)
len(embeded_query), embeded_query[:15]

(1536,
 [-0.027967285621545507,
  0.007272935520456191,
  0.00372656855753541,
  -0.036588130280438405,
  -0.030850511135379583,
  0.01670829110761046,
  -0.006833243722728746,
  -0.011114833238243343,
  -0.0013605219229251401,
  -0.02120612253666261,
  0.03330125095949351,
  0.025934610889234464,
  -0.02910615895445195,
  -0.0020831302940467426,
  -0.019865422300200757])

In [ ]:
result = index.query(
    # namespace="ns1",
    vector=embeded_query,
    top_k=3,
    include_values=True
)

In [ ]:
len(result.matches), \
result.matches[0].id, \
result.matches[0].score, \
result.matches[0].values[:15]

(3,
 '6127f943-c2e2-4a09-9d60-357eec482557',
 0.796478271,
 [-0.0240785182,
  0.0146332774,
  0.012422544,
  -0.0345504135,
  0.0149207413,
  0.00992434658,
  -0.0147154108,
  0.00101125659,
  0.00319119077,
  -0.00310905813,
  0.0283357203,
  0.0291296691,
  0.00343587715,
  0.00145614112,
  -0.00544127962])

In [ ]:
index.fetch([result.matches[0].id]) \
  .vectors[result.matches[0].id]['metadata']["text"]

'Decoder: The decoder is also composed of a stack of N= 6identical layers. In addition to the two\nsub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head\nattention over the output of the encoder stack. Similar to the encoder, we employ residual connections\naround each of the sub-layers, followed by layer normalization. We also modify the self-attention\nsub-layer in the decoder stack to prevent positions from attending to subsequent positions. This'

### ChromaDB

In [ ]:
!pip install -q chromadb openai langchain tiktoken langchain-openai

In [ ]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
!unzip -q new_articles.zip -d new_articles
!rm -f new_articles/05-03** new_articles/05-04** new_articles/05-05**

In [ ]:
import os
openai_api_key = "" # @param {type:"string"}
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [ ]:
loader = DirectoryLoader("./new_articles", glob="./*.txt", loader_cls=TextLoader)

In [ ]:
document = loader.load()

len(document), document[0]

(6,
 Document(page_content='Welcome back to This Week in Apps, the weekly TechCrunch series that recaps the latest in mobile OS news, mobile applications and the overall app economy.\n\nThe app economy in 2023 hit a few snags, as consumer spending last year dropped for the first time by 2% to $167 billion, according to data.ai’s “State of Mobile” report. However, downloads are continuing to grow, up 11% year-over-year in 2022 to reach 255 billion. Consumers are also spending more time in mobile apps than ever before. On Android devices alone, hours spent in 2022 grew 9%, reaching 4.1 trillion.\n\nThis Week in Apps offers a way to keep up with this fast-moving industry in one place with the latest from the world of apps, including news, updates, startup fundings, mergers and acquisitions, and much more.\n\nDo you want This Week in Apps in your inbox every Saturday? Sign up here: techcrunch.com/newsletters\n\nTop Stories\n\nDorsey criticizes Twitter, Musk on the alternative social networ

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text = text_splitter.split_documents(document)

In [ ]:
len(text), text[0]

(80,
 Document(page_content='Welcome back to This Week in Apps, the weekly TechCrunch series that recaps the latest in mobile OS news, mobile applications and the overall app economy.\n\nThe app economy in 2023 hit a few snags, as consumer spending last year dropped for the first time by 2% to $167 billion, according to data.ai’s “State of Mobile” report. However, downloads are continuing to grow, up 11% year-over-year in 2022 to reach 255 billion. Consumers are also spending more time in mobile apps than ever before. On Android devices alone, hours spent in 2022 grew 9%, reaching 4.1 trillion.\n\nThis Week in Apps offers a way to keep up with this fast-moving industry in one place with the latest from the world of apps, including news, updates, startup fundings, mergers and acquisitions, and much more.\n\nDo you want This Week in Apps in your inbox every Saturday? Sign up here: techcrunch.com/newsletters\n\nTop Stories\n\nDorsey criticizes Twitter, Musk on the alternative social netwo

#### Creating ChromaDB

In [ ]:
from langchain import embeddings
from langchain_openai import OpenAIEmbeddings
persist_directory = 'db'

embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents=text,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
# To persiste the data to disk
vectordb.persist()

In [ ]:
vectordb = None
# Loading the persisted database from disk
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [ ]:
vectordb

#### Retrieval

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents(
    "When did Google rolled out passkeys to Google Account users globally"
)

docs

[Document(page_content='Google ditches passwords for passkeys: This week Google rolled out passkeys to Google Account users globally, roughly a year after the company — alongside Apple, Microsoft and the FIDO Alliance — announced a partnership to broadly advance passkey adoption. With passkeys, users’ authentication synchronizes across devices through the cloud using cryptographic key pairs, allowing them to sign in to websites and apps using the same biometrics or screen-lock PIN they use to unlock their devices.\n\nMicrosoft debuts Pegasus: Microsoft this week announced that it’ll extend the Startup Founders Hub, its self-service platform that provides founders with free resources, including Azure credits, with a new incubator program called the Pegasus Program. Pegasus will select startups with products that “fill a market need” and give them up to $350,000 in Azure, GitHub and LinkedIn credits plus backing from advisors, as well as “access to the best Microsoft tech,” Microsoft say

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k":2})
docs = retriever.get_relevant_documents(
    "What did Amazon CEO Andy Jassy during the company’s first-quarter earnings call"
)

docs

[Document(page_content='A new LLM for Alexa: Amazon is building a more “generalized and capable” large language model to power Alexa, said Amazon CEO Andy Jassy during the company’s first-quarter earnings call this week. He added that although Amazon has had an LLM powering Alexa, Amazon is working on one that’s more capable than the current one.\n\naudio', metadata={'source': 'new_articles/05-06-amazon-launches-free-channels-check-marks-come-to-gmail-and-openai-raises-more-moolah.txt'}),
 Document(page_content='A new LLM for Alexa: Amazon is building a more “generalized and capable” large language model to power Alexa, said Amazon CEO Andy Jassy during the company’s first-quarter earnings call this week. He added that although Amazon has had an LLM powering Alexa, Amazon is working on one that’s more capable than the current one.\n\naudio', metadata={'source': 'new_articles/05-06-amazon-launches-free-channels-check-marks-come-to-gmail-and-openai-raises-more-moolah.txt'})]

#### Making a Chain

In [ ]:
llm = OpenAI()
llm

OpenAI(client=<openai.resources.completions.Completions object at 0x78d8651c86a0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x78d8651cb610>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
def process_llm_response(llm_response):
  print(llm_response['result'])
  print('\nSources:')
  for source in llm_response['source_documents']:
    print(source.metadata["source"])

In [ ]:
# Usage Example
query = "What was the Apple and Google joint announcement?"
process_llm_response(qa_chain(query))

 They announced that they will be working together to draft a specification that would alert users in the case of unwanted tracking from Bluetooth devices.

Sources:
new_articles/05-06-this-week-in-apps-apple-and-google-team-up-on-trackers-google-i-o-preview-apps-hit-newfronts.txt
new_articles/05-06-this-week-in-apps-apple-and-google-team-up-on-trackers-google-i-o-preview-apps-hit-newfronts.txt


#### Removing ChromaDB

In [ ]:
# Zipping to save the DB
!zip -r db.zip ./db

  adding: db/ (stored 0%)
  adding: db/chroma.sqlite3 (deflated 45%)
  adding: db/3b591d91-cf4f-4043-9482-ec2227d3efd2/ (stored 0%)
  adding: db/3b591d91-cf4f-4043-9482-ec2227d3efd2/header.bin (deflated 61%)
  adding: db/3b591d91-cf4f-4043-9482-ec2227d3efd2/link_lists.bin (stored 0%)
  adding: db/3b591d91-cf4f-4043-9482-ec2227d3efd2/length.bin (deflated 84%)
  adding: db/3b591d91-cf4f-4043-9482-ec2227d3efd2/data_level0.bin (deflated 100%)


In [ ]:
vectordb.delete_collection()
vectordb.persist()

## Llama

### Llama CPP Python

In [1]:
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 16.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.59-cp310-cp310-manylinux_2_35_x86_64.whl size=38789211 sha256=7e6d11a30a6666e5f59efdc3bc6008418edac2761396e834ec7d5d8ed67e81a8
  Stored in directory: /root/.cache/pip/wheels/67/d1/01/78c9a09d3b1f348adbc9dad22b238dba3e20c12479986b8efa
Successfully built llama-cpp-python


In [2]:
model_name = 'TheBloke/Llama-2-13B-chat-GGUF'
model_basename= 'llama-2-13b-chat.Q5_K_M.gguf'

In [3]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [4]:
model_path = hf_hub_download(
    repo_id=model_name,
    filename=model_basename
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [5]:
model_path

'/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf'

In [6]:
!nvidia-smi

Fri Apr  5 03:00:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
# GPU
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU Cores
    n_batch=1024, # Should be between 1 and n_ctx, consider the amount of VRAM in GPU
    n_gpu_layers=-1 # Change this value based on your model and your GPU VRAM pool
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_

In [8]:
prompt="Write a linear regression code"
prompt_template=f'''SYSTEM: You are a helpful assistant.

USER: {prompt}

ASSISTANT:
'''

print(prompt_template)

SYSTEM: You are a helpful assistant.

USER: Write a linear regression code

ASSISTANT:



In [9]:
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=256,
    temperature=0.5,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=150,
    echo=True
)


llama_print_timings:        load time =     892.48 ms
llama_print_timings:      sample time =     156.96 ms /   256 runs   (    0.61 ms per token,  1631.02 tokens per second)
llama_print_timings: prompt eval time =     892.37 ms /    27 tokens (   33.05 ms per token,    30.26 tokens per second)
llama_print_timings:        eval time =   12067.04 ms /   255 runs   (   47.32 ms per token,    21.13 tokens per second)
llama_print_timings:       total time =   13926.02 ms /   282 tokens


In [10]:
response

{'id': 'cmpl-2d9e8f48-31fa-40b8-8071-68055287c39b',
 'object': 'text_completion',
 'created': 1712286064,
 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf',
 'choices': [{'text': 'SYSTEM: You are a helpful assistant.\n\nUSER: Write a linear regression code\n\nASSISTANT:\n\nHere is an example of how you could write a linear regression code in Python using scikit-learn library:\n```\nimport pandas as pd\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.model_selection import train_test_split\n\n# Load the data\ndf = pd.read_csv(\'data.csv\')\n\n# Split the data into training and testing sets\nX_train, X_test, y_train, y_test = train_test_split(df.drop(\'target\', axis=1), df[\'target\'], test_size=0.2)\n\n# Create a linear regression object and fit it to the training data\nreg = LinearRegression()\nreg.fit(X_train, y_train)\n\n# Make predictions on the testing dat

In [13]:
print(response['choices'][0]['text'])

SYSTEM: You are a helpful assistant.

USER: Write a linear regression code

ASSISTANT:

Here is an example of how you could write a linear regression code in Python using scikit-learn library:
```
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Load the data
df = pd.read_csv('data.csv')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2)

# Create a linear regression object and fit it to the training data
reg = LinearRegression()
reg.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = reg.predict(X_test)

# Evaluate the model using mean squared error
mse = ((y_test - y_pred) ** 2).mean()
print('Mean squared error:', mse)
```
This code assumes that you have a CSV file called "data.csv" containing the data, with


### Llama with Langchain

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [14]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [1]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

In [2]:
model="meta-llama/Llama-2-7b-hf"

In [3]:
tokenizer=AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [4]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [6]:
# This LLM can be used with all other langchain functions
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [12]:
prompt="What is a good name for a tech company"

In [13]:
print(llm.invoke(prompt))

What is a good name for a tech company?
What is a good name for a tech company?
What is the most successful tech company?
What is the most successful tech company in the world?
What are the top 10 tech companies?
What is the best tech company to work for?
What is the best tech company to work for?
What is the best tech company in the world?
What is the most successful company in the world?
What is the most successful company in the world?
What is the best company in the world?
What is the best tech company to work for?
What is the best tech company to work for?
What is the best tech company to work for?
What is the best tech company to work for?
What is the best tech company to work for?
What is the best tech company to work for?What is the best tech company to work for?
What is the best tech company to work for?What is the best tech company to work for?What is the best tech company to work for?What is the best tech company to work for?What is the best tech company to work for?What is 